In [ ]:
%cd /content/drive/MyDrive/Github/floread
!git checkout feature_sentiment-analysis

/content/drive/MyDrive/Github/floread
M	sentiment-analysis/train koelectra-v2.ipynb
M	sentiment-analysis/visualize.ipynb
Already on 'feature_sentiment-analysis'
Your branch is up to date with 'origin/feature_sentiment-analysis'.


In [ ]:
!pip install transformers
import pandas as pd

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset

import transformers
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 39.9 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/Github/floread/sentiment-analysis

train_set = pd.read_csv('data/병합데이터셋-v3.csv', index_col=0)
train_set.sample(n=5)

/content/drive/MyDrive/Github/floread/sentiment-analysis


,sentence,emotion
93497,"나 없는 동안, 결재 못한 일이 산처럼 쌓였어. 이러다 우리회사 신용 떨어진다고, ...",중립
58345,비교할사람이랑 비교를 해야지;;;;,불안
25482,이번에 남편 생일이라 선물을 준비해서 줬어. 남편이 좋아하니 나도 기분이 좋아.,행복
18583,놀다가 지쳐서 쉬던 중이야. 넌? 선물 사러 온 거 아니었어?,중립
6946,항상 응원하고 있습니다~~^^,행복


In [ ]:
# 감정을 정수 라벨로 변경
emotions = {'행복': 0, '불안': 1, '놀람': 2, '슬픔': 3, '분노': 4, '중립': 5}
train_set['emotion'] = train_set.emotion.map(emotions)

train_set.sample(n=5)

,sentence,emotion
31237,요즘 건강을 위해서 공원으로 운동 다니는데 괜히 남들 시선이 신경 쓰이는 거 있지.,2
71302,남편이 퇴직해서 연금을 받기 시작했는데 문제가 생겼어.,3
29376,작작좀 해쳐먹지.,4
121954,정규직이 아니라 불안하고 급여가 너무 적어서 고민이야.,1
73011,지겹다. 지겨워서 빨리 끝내고 쉬고 싶어. 다음에 맘 편하게 만나자.,4


In [ ]:
# torch.utils.data.Dataset 상속
class mscDataset(Dataset):  #multi-label sentiment classification
  def __init__(self, csv_file):
    # 결측치 제거
    self.dataset = csv_file
    # 중복값 제거
    self.dataset.drop_duplicates(subset=['sentence'], inplace=True)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

    print(self.dataset.describe())

#   def __init__(self, csv_file):
#     self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")
#     print(self.dataset.describe())

  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 0:2].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=256,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

---

In [ ]:
train_data, test_data = train_test_split(train_set, test_size=0.2, random_state=0)
train_dataset = mscDataset(train_data)
test_dataset = mscDataset(test_data)

             emotion
count  108617.000000
mean        3.113343
std         1.738215
min         0.000000
25%         2.000000
50%         3.000000
75%         5.000000
max         5.000000
            emotion
count  27159.000000
mean       3.115800
std        1.739844
min        0.000000
25%        2.000000
50%        3.000000
75%        5.000000
max        5.000000


In [ ]:
#모델 불러오기
device = torch.device("cuda:0") #(colab)그냥 "cuda"로 하면 애러
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=6).to(device)

# 하이퍼 파라미터
epochs = 8
batch_size = 16
optimizer = AdamW(model.parameters(), lr=5e-5)

#데이터 로더
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

In [ ]:
# 학습
losses = []
accuracies = []

for i in range(1, epochs+1):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad()
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    _, predicted = torch.max(y_pred, 1)
    correct += (predicted == y_batch).sum()
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  model.eval()
  test_correct = 0
  test_total = 0
  for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    _, predicted = torch.max(y_pred, 1)
    test_correct += (predicted == y_batch).sum()
    test_total += len(y_batch)
  print("epoch",i,"test acc", test_correct.float() / test_total)

  0%|          | 0/6789 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Batch Loss: 172.3260395526886 Accuracy: tensor(0.3219, device='cuda:0')
Batch Loss: 328.97081434726715 Accuracy: tensor(0.3616, device='cuda:0')
Batch Loss: 475.20121455192566 Accuracy: tensor(0.3919, device='cuda:0')
Batch Loss: 621.0231098532677 Accuracy: tensor(0.4025, device='cuda:0')
Batch Loss: 761.1147382855415 Accuracy: tensor(0.4165, device='cuda:0')
Batch Loss: 898.2895245552063 Accuracy: tensor(0.4261, device='cuda:0')
Batch Loss: 1026.390774667263 Accuracy: tensor(0.4364, device='cuda:0')
Batch Loss: 1158.7199918031693 Accuracy: tensor(0.4422, device='cuda:0')
Batch Loss: 1280.8191049098969 Accuracy: tensor(0.4509, device='cuda:0')
Batch Loss: 1403.7280105948448 Accuracy: tensor(0.4589, device='cuda:0')
Batch Loss: 1526.834138393402 Accuracy: tensor(0.4643, device='cuda:0')
Batch Loss: 1647.1571494340897 Accuracy: tensor(0.4695, device='cuda:0')
Batch Loss: 1769.0262551307678 Accuracy: tensor(0.4746, device='cuda:0')
Batch Loss: 1891.5523330569267 Accuracy: tensor(0.4773, d

  0%|          | 0/1698 [00:00<?, ?it/s]

epoch 1 test acc tensor(0.6683, device='cuda:0')


...

epoch 7 test acc tensor(0.6834, device='cuda:0')


  0%|          | 0/6789 [00:00<?, ?it/s]

Batch Loss: 53.46692967414856 Accuracy: tensor(0.7906, device='cuda:0')
Batch Loss: 107.52370436489582 Accuracy: tensor(0.7987, device='cuda:0')
Batch Loss: 157.48024325072765 Accuracy: tensor(0.8033, device='cuda:0')
Batch Loss: 208.94569918513298 Accuracy: tensor(0.8070, device='cuda:0')
Batch Loss: 260.57958071678877 Accuracy: tensor(0.8088, device='cuda:0')
Batch Loss: 312.29075402766466 Accuracy: tensor(0.8101, device='cuda:0')
Batch Loss: 362.1628898307681 Accuracy: tensor(0.8118, device='cuda:0')
Batch Loss: 419.44005943089724 Accuracy: tensor(0.8087, device='cuda:0')
Batch Loss: 474.0206001326442 Accuracy: tensor(0.8076, device='cuda:0')
Batch Loss: 529.3866503313184 Accuracy: tensor(0.8078, device='cuda:0')
Batch Loss: 585.1697023585439 Accuracy: tensor(0.8075, device='cuda:0')
Batch Loss: 639.2341019883752 Accuracy: tensor(0.8075, device='cuda:0')
Batch Loss: 698.6778594627976 Accuracy: tensor(0.8056, device='cuda:0')
Batch Loss: 753.5861591473222 Accuracy: tensor(0.8060, dev

  0%|          | 0/1698 [00:00<?, ?it/s]

epoch 8 test acc tensor(0.6803, device='cuda:0')


In [ ]:
losses, accuracies

([7368.098013758659,
  6061.961144074798,
  5594.054321825504,
  5206.550875872374,
  4798.972378477454,
  4470.850164487958,
  4127.805078975856,
  3780.395183928311],
 [tensor(0.5959, device='cuda:0'),
  tensor(0.6724, device='cuda:0'),
  tensor(0.6983, device='cuda:0'),
  tensor(0.7181, device='cuda:0'),
  tensor(0.7427, device='cuda:0'),
  tensor(0.7606, device='cuda:0'),
  tensor(0.7779, device='cuda:0'),
  tensor(0.7978, device='cuda:0')])

In [ ]:
torch.save(model, 'model/koelectra-v3.pt')

In [ ]:
train_data, test_data = train_test_split(train_set, test_size=0.2, random_state=0)
train_dataset = mscDataset(train_data)
test_dataset = mscDataset(test_data)

In [ ]:
#모델 불러오기
device = torch.device("cuda:0") #(colab)그냥 "cuda"로 하면 애러
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", num_labels=6).to(device)

# 하이퍼 파라미터
epochs = 8
batch_size = 16
optimizer = AdamW(model.parameters(), lr=1e-6)

#데이터 로더
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)